<a href="https://colab.research.google.com/github/rajaih/Perci-rajai/blob/master/MOST_WORKING_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistral_ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.9 MB/s eta 0:00:00


In [ ]:
#!pip install mcp_util

ERROR: Could not find a version that satisfies the requirement mcp_util (from versions: none)
ERROR: No matching distribution found for mcp_util


In [10]:
import sys
##import mcp_util
#import OS
sys.path.append('../')
#from mcp_util.config import FastMCP, Context
#from mcp_util.logger import get_logger, current_tool_name
#from mcp_util.constants import SERVER_NAME
#import OS
import base64
import tempfile
import time
from typing import List, Dict,Any, Annotated
from pydantic import BaseModel, Field, SecretStr
from mistralai import Mistral
import base64
import json
api_key1="g5eRGlSC6sETGwyzipp5qM1bBEthhO84"
client = Mistral(api_key=api_key1)
class JSON_Label_List(BaseModel):
  json_:List = Field(description="List of label and value pair extracted from the text")
def base64_to_temp_pdf(pdf_base64: str) -> str:
    """Convert base64 data to temporary PDF file."""
    base64_data = pdf_base64
    try:
      if base64_data.startswith('data:'):
        base64_data = base64_data.split(',')[1]
      #decode base 64 data
      pdf_data = base64.b64decode(base64_data)
      #create temporaty file
      temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
      temp_file.write(pdf_data)
      temp_file.close()
      return temp_file.name
    except Exception as e:
      print(f"Error decoding base64 data: {e}")
      return None



def mistral_extract_document_entities(
    pdf_base64: Annotated[str, "Base64 encoded PDF file data to etract entities from"],
    entities: Annotated[List[str], "List of entities to extract from the PDF"] =[],
) -> Dict[str,Any]:
    """
    Extract entities from a PDF using MistralAI
    Args:
        base64_data: Base64 encoded PDF file data to etract entities from
        entities: List of entities to extract, each with 'name' and 'description' keys
    Returns:
        Dictionary containing the extracted entities
    """
    base64_data = pdf_base64
    temp_pdf_file = None
    try:
        result1 = client.ocr.process(
            model = "mistral-ocr-latest",
            document = {
                "type": "document_url",
                "data": base64_data,
                "document_url": f"data:application/pdf;base64,{pdf_base64}"
            },
            include_image_base64 = True
        )
        extracted_text = result1.model_dump_json()
        prompt = f"""
        You are an expert in extracting information.
        The information you need to extract is : {entities} for all pages
        note - Output should be in json array format(entity as key and value as value).
        Document Text : {extracted_text}
        """
        messagesP = [

            {
                "role": "user",
                "content": prompt
            }
        ]
    except Exception as e:
        print(f"Error during OCR processing: {e}")
        return None # Return None if OCR fails
    finally:
          #if temp_pdf_path and os:
           # os.remove(temp_pdf_file)
            pass # Use pass here if you don't want to return

    chat_response = client.chat.complete(
        model = "mistral-small-latest",
        messages = messagesP
    )
    print("DICTONARY")
    messages1 = dict(chat_response)
    print(messages1)
    raw_text = chat_response.choices[0].message.content
    print(raw_text)
    file1 = "cresponse.json"
    try:
      with open(file1, "w", encoding ="utf-8") as file:
        file.write(raw_text)
        print(f"chat response successfully saved to '{file1}'")
    except IOError as e:
          print(f"Error writing to file '{file1}': {e}")
          return None
    print(file1)
    messages2 = {k: v.replace("\n","") if isinstance(v, str) else v for k, v in messages1.items()}
    print(messages2)
    print(" I am message2")
    extracted_entities =[]
    # The raw_text is a string, not a list, so iterating over it character by character is incorrect.
    # Assuming raw_text contains a JSON string representing a list of dictionaries:
    try:
        raw_text_list = json.loads(raw_text)
        for row in raw_text_list:
            new_row = {key: row[key] for key in entities if key in row}
            extracted_entities.append(new_row)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from raw_text: {e}")
        # Handle the case where raw_text is not a valid JSON list
        return {
            "success": False,
            "error": "Invalid JSON format in chat response",
            "source_text": extracted_text
        }

    return{
        "success": True,
        "extracted_entities" : extracted_entities, # Return the processed list
        "source_text": extracted_text,
       # "source_file": temp_pdf_file

    }
    # The finally block below is misplaced and will execute before the return above.
    # It also has an unnecessary return None which would prevent the function from returning the actual results.
    # I will remove this misplaced finally block.
#finally:
      #if temp_pdf_path and os:
       # os.remove(temp_pdf_file)
        #return None



pdf_path = "/content/OCR_DOC3.pdf"
encoded = ""
with open(pdf_path, "rb") as f:
    encoded = base64.b64encode(f.read()).decode("utf-8")
#return encoded # This return is outside the function and will cause an error. Commenting it out.
results  = mistral_extract_document_entities(encoded, ["Test","Marks"])
print(results)

DICTONARY
{'id': 'b3fb04ca269c4bbfbcfab7323cdb841c', 'object': 'chat.completion', 'model': 'mistral-small-latest', 'usage': UsageInfo(prompt_tokens=370, completion_tokens=134, total_tokens=504, prompt_audio_seconds=Unset()), 'created': 1763605999, 'choices': [ChatCompletionChoice(index=0, message=AssistantMessage(content='Here is the extracted information in JSON array format for the entities \'Test\' and \'Marks\' from all pages:\n\n```json\n[\n    {"Test": "MID", "Marks": 80},\n    {"Test": "FINAL", "Marks": 90},\n    {"Test": "HALFYEAR", "Marks": 100},\n    {"Test": "FINAL", "Marks": 90},\n    {"Test": "FINAL", "Marks": 90}\n]\n```\n\nNote: Page 4 was skipped as it only contained a dot (.) and no relevant information.', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]}
Here is the extracted information in JSON array format for the entities 'Test' and 'Marks' from all pages:

```json
[
    {"Test": "MID", "Marks": 80},
    {"Test": "FINAL", "Marks": 90},
    {

In [ ]:
def mistral_extract_document_entities(
    pdf_base64: Annotated[str, "Base64 encoded PDF file data to etract entities from"],
    entities: Annotated[List[str], "List of entities to extract from the PDF"] =[],
) -> Dict[str,Any]:
    """
    Extract entities from a PDF using MistralAI
    Args:
        base64_data: Base64 encoded PDF file data to etract entities from
        entities: List of entities to extract, each with 'name' and 'description' keys
    Returns:
        Dictionary containing the extracted entities
    """
    base64_data = pdf_base64
    temp_pdf_file = None
    try:
        new_var = client.ocr.process(
            model = "mistral-ocr-latest",
            document = {
                "type": "document_url",
                "data": base64_data,
                "document_url": f"data:application/pdf;base64,{pdf_base64}"
            },
            include_image_base64 = True
        )
        result1 = new_var
        extracted_text = result1.model_dump_json()
        prompt = f"""
        You are an expert in extracting information.
        The information you need to extract is : {entities} for all pages
        note - Output should be in json array format(entity as key and value as value).
        Document Text : {extracted_text}
        """
        messagesP = [

            {
                "role": "user",
                "content": prompt
            }
        ]
    except Exception as e:
        print(f"Error during OCR processing: {e}")
        return None # Return None if OCR fails
    finally:
          #if temp_pdf_path and os:
           # os.remove(temp_pdf_file)
            pass # Use pass here if you don't want to return

    chat_response = client.chat.complete(
        model = "mistral-small-latest",
        messages = messagesP
    )
    print("DICTONARY")
    messages1 = dict(chat_response)
    print(messages1)
    raw_text = chat_response.choices[0].message.content
    print(raw_text)
    file1 = "cresponse.json"
    try:
      with open(file1, "w", encoding ="utf-8") as file:
        file.write(raw_text)
        print(f"chat response successfully saved to '{file1}'")
    except IOError as e:
          print(f"Error writing to file '{file1}': {e}")
          return None
    print(file1)
    messages2 = {k: v.replace("\n","") if isinstance(v, str) else v for k, v in messages1.items()}
    print(messages2)
    print(" I am message2")
    extracted_entities =[]
    # The raw_text is a string, not a list, so iterating over it character by character is incorrect.
    # Assuming raw_text contains a JSON string representing a list of dictionaries:
    try:
        raw_text_list = json.loads(raw_text)
        for row in raw_text_list:
            new_row = {key: row[key] for key in entities if key in row}
            extracted_entities.append(new_row)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from raw_text: {e}")
        # Handle the case where raw_text is not a valid JSON list
        return {
            "success": False,
            "error": "Invalid JSON format in chat response",
            "source_text": extracted_text
        }

    return{
        "success": True,
        "extracted_entities" : extracted_entities, # Return the processed list
        "source_text": extracted_text,
       # "source_file": temp_pdf_file

    }
    # The finally block below is misplaced and will execute before the return above.
    # It also has an unnecessary return None which would prevent the function from returning the actual results.
    # I will remove this misplaced finally block.
#finally:
      #if temp_pdf_path and os:
       # os.remove(temp_pdf_file)
        #return None



pdf_path = "/content/OCR_DOC3.pdf"
encoded = ""
with open(pdf_path, "rb") as f:
    encoded = base64.b64encode(f.read()).decode("utf-8")
#return encoded # This return is outside the function and will cause an error. Commenting it out.
results  = mistral_extract_document_entities(encoded, ["name", "value"])
print(results)

SDKError: API error occurred: Status 429. Body: {"object":"error","message":"Service tier capacity exceeded for this model.","type":"service_tier_capacity_exceeded","param":null,"code":"3505"}